In [ ]:
import sys

sys.path.append('..')

from src.models.miniconvnext import MiniConvNeXt

import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
import numpy as np

from src.trainer import Trainer
from src.dataset import HumanPosesDataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as T
import torch

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

In [ ]:
from torchvision import transforms

mean = [0.4638, 0.4522, 0.4148]
std = [0.2222, 0.2198, 0.2176]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.2), ratio=(0.3, 3.3), value='random')
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

In [ ]:
CSV_PATH = Path("../data/human_poses_data/train_answers.csv")
TRAIN_DIR = Path("../data/human_poses_data/img_train")

df = pd.read_csv(CSV_PATH)

train_ids, val_ids = train_test_split(
    df['img_id'].values,
    test_size=0.2,
    stratify=df['target_feature'],
    random_state=42
)

train_df = df[df['img_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)

train_dataset = HumanPosesDataset(
    data_df=train_df,
    img_dir=TRAIN_DIR,
    transform=train_transform,
)

val_dataset = HumanPosesDataset(
    data_df=val_df,
    img_dir=TRAIN_DIR,
    transform=val_transform,
)



train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
num_classes = len(np.unique(df['target_feature']))
print(f"Количество классов: {num_classes}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

In [ ]:
from mmpretrain import get_model

model = get_model(
    'tinyvit-5m_3rdparty_in1k',
    pretrained=False,
    head=dict(num_classes=16)
)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(train_loader),
    epochs=NUM_EPOCH,
    pct_start=0.1,
    anneal_strategy='cos',
    div_factor=10.0,
    final_div_factor=500
)


criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

scaler = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.3)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="16vit_1",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
from src.utils import load_best_model

load_best_model(model, 'checkpoints/16vit_1_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

scaler = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.5, p_mixup=0.2)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="16vit_2",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
from src.utils import load_best_model

load_best_model(model, 'checkpoints/16vit_2_best.pth', device) #для восрпоиводимости: сверху было 50 эпох,а не 75

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 75

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-6
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

scaler = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=0.5, p_mixup=0.2)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=nn.CrossEntropyLoss(label_smoothing=0.1),
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="16vit_3",
    use_wandb=True,
    seed=42,
)

history = trainer.train()

In [ ]:
load_best_model(model, f'checkpoints/16vit_3_best.pth', device)

In [ ]:
from torch.amp import GradScaler

NUM_EPOCH = 25

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH,
    eta_min=1e-7
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

scaler = GradScaler()

In [ ]:
load_best_model(model, f'checkpoints/16vit_4_best.pth', device)

# Сабмит

In [ ]:
load_best_model(model, f'checkpoints/16vit_4_best.pth', device)

In [ ]:
from pathlib import Path
import pandas as pd

TEST_DIR = Path("../data/human_poses_data/img_test")

test_ids = [f.stem for f in TEST_DIR.glob("*.jpg")]
test_df = pd.DataFrame({"img_id": test_ids})


test_dataset = HumanPosesDataset(
    data_df=test_df,
    img_dir=TEST_DIR,
    transform=val_transform,
    preload=False,
    mode='test',
)


test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
from src.utils import make_submission

make_submission(model, test_loader, device, train_dataset.index_to_class)

In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"

In [ ]:
from src.test_dataset_for_tta import TestDataset, make_submission_with_tta
from torch.utils.data import DataLoader

def pil_collate(batch):
    images, ids = zip(*batch)
    return list(images), list(ids)


test_image_paths = list(TEST_DIR.glob("*.jpg"))
test_ids = [int(p.stem) for p in test_image_paths]

test_dataset = TestDataset(test_image_paths, test_ids)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
    collate_fn=pil_collate,
    pin_memory=True
)




In [ ]:
make_submission_with_tta(model, test_loader, device, train_dataset.index_to_class)

In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"